# 1주일 간 네이버 금융 뉴스 내용을 수집하여 txt로 저장

In [1]:
# 1번 일주일 간 뉴스 내용 파일 저장
import requests
from bs4 import BeautifulSoup as BS
import os

head = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36 Edg/93.0.961.38'
}
if not os.path.isdir("./article"): os.mkdir("./article")
    
def get_article(bs):
    temp_article_list = []
    # 페이지 기사 로드
    for content in bs.find('div', class_ = 'list_body newsflash_body').findAll('dt', class_ = lambda x: not x == 'photo'):
        # 주소목록 추출(index 추후 확인 필요)
        temp_article_list.append(content.findAll('a')[0]['href'])
    return temp_article_list

def get_last_page_text(bs):
    return bs.find('div', class_ = 'content').findAll('a', class_ = 'nclicks(fls.page)')[::-1][0].text

def get_article_text(bs):
    content = bs.find('div', class_ = '_article_body_contents')
    content.script.decompose()
#     content.span.decompose()
    return content.text.strip()

last_page_text = ''
article_date = ['20210909', '20210908', '20210907', '20210906', '20210905', '20210904', '20210903']

for date in article_date:
    now_page = 1
    article_list = []
    while(True):
        r = requests.get(f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date={date}&page={now_page}', headers=head)
        bs = BS(r.text)
        if now_page % 10 == 1: last_page_text = get_last_page_text(bs)
        article_list.append(get_article(bs))
        # ex) 21페이지에서 마지막인 경우 현재 페이지는 선택할 수 없으므로 하이퍼링크가 존재하지 않고 이전 페이지의 하이퍼링크만 존재하여 오류 발생
        if (last_page_text != '다음' and last_page_text.isdigit() and now_page == int(last_page_text)) or last_page_text == '이전' : break
        now_page += 1

    if not os.path.isdir(f'./article/{date}'): os.mkdir(f'./article/{date}')

    for idx1, page_article in enumerate(article_list):
        for idx2, article_url in enumerate(page_article):
            r = requests.get(article_url, headers=head)
            bs = BS(r.text)
            # 리스트에 노출되지만 비공개로 전환된 기사의 경우 error 메세지 출력
            if bs.find('div', class_ = 'error') is None:
                with open(f'./article/{date}/{idx1}-{idx2}.txt', 'w', encoding='utf-8') as f:
                    f.write(get_article_text(bs))

C:\Users\Cheon\anaconda3\lib\site-packages\bs4\__init__.py:177: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 194 of the file C:\Users\Cheon\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  warnings.warn(self.NO_PARSER_SPECIFIED_WARNING % dict(


In [161]:
# r = requests.get('https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=20210909&page=19', headers=head)
#https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=259&sid1=101&date=20210910&page=4
# r = requests.get("https://news.naver.com/main/list.naver?mode=LS2D&sid2=263&sid1=101&mid=shm&date=20210910&page=19", headers=head)

In [162]:
# bs = BS(r.text)

In [114]:
# 특정일 기사 수집
# bs.find('div', class_ = 'content').findAll('a')
# 페이지 수 조회
# last_page_text = bs.find('div', class_ = 'content').findAll('a', class_ = 'nclicks(fls.page)')[::-1][0].text
# 다음이 출력되는 시점은 10페이지를 넘기는 시점이므로 비교할 별도의 비교 불필요
# if last_page_text == '다음': last_page_text = bs.find('div', class_ = 'content').findAll('a', class_ = 'nclicks(fls.page)')[::-1][1].text

# 수집한 데이터를 분석하여 트랜드 키워드 추출

In [55]:
from konlpy.tag import Komoran
# from nltk.corpus import stopwords 
# from nltk.tokenize import word_tokenize
import os
import operator

In [4]:
with open("./article/20210909/0-0.txt", "r", encoding='utf-8') as f:
    text = f.read()

In [64]:
komoran = Komoran()
total_dict = {}
for folder in article_folder_list:
    temp_dict = {}
    for file in os.listdir("./article/" + folder):
        with open(f'./article/{folder}/{file}', "r", encoding='utf-8') as f:
            text = f.read()
        for x in komoran.pos(text):
            if x[1] == "NNP" or x[1] == "NNG" or x[1] == 'VV':
                if temp_dict.get(x[0]) is None:
                    temp_dict[x[0]] = 1
                else: temp_dict[x[0]] += 1
    total_dict[folder] = sorted(temp_dict.items(), key=operator.itemgetter(1), reverse=True)

In [65]:
total_dict

{'20210903': [('금융', 1591),
  ('대출', 1014),
  ('금리', 778),
  ('있', 613),
  ('하', 545),
  ('은행', 424),
  ('위하', 417),
  ('따르', 376),
  ('통하', 376),
  ('대하', 366),
  ('위원장', 344),
  ('포인트', 321),
  ('말', 314),
  ('기준', 290),
  ('서울', 286),
  ('기자', 261),
  ('서비스', 257),
  ('시장', 256),
  ('인상', 255),
  ('관리', 248),
  ('투자', 244),
  ('자산', 238),
  ('지원', 236),
  ('받', 232),
  ('상품', 232),
  ('정책', 228),
  ('되', 225),
  ('밝히', 215),
  ('신용', 214),
  ('오르', 196),
  ('코로나', 187),
  ('고객', 186),
  ('증가', 185),
  ('경우', 180),
  ('관계자', 175),
  ('한국은행', 171),
  ('적금', 171),
  ('회동', 169),
  ('주택', 165),
  ('지나', 165),
  ('열', 164),
  ('고승', 164),
  ('범', 164),
  ('가계부채', 162),
  ('이', 160),
  ('당국', 157),
  ('가계', 157),
  ('위', 156),
  ('소비자', 156),
  ('이상', 154),
  ('자금', 153),
  ('국민은행', 153),
  ('올해', 152),
  ('법', 150),
  ('관련', 149),
  ('이주열', 149),
  ('위원회', 146),
  ('이날', 145),
  ('19', 145),
  ('성장', 143),
  ('중단', 140),
  ('기관', 140),
  ('기업', 140),
  ('지난해', 139),
  ('업무', 138),
  ('규제

In [37]:
for x in article_folder_list:
    print(f'{x}{os.listdir("./article/" + x)}')

20210903['0-1.txt', '0-10.txt', '0-11.txt', '0-12.txt', '0-13.txt', '0-14.txt', '0-15.txt', '0-16.txt', '0-17.txt', '0-18.txt', '0-19.txt', '0-2.txt', '0-3.txt', '0-4.txt', '0-6.txt', '0-7.txt', '0-8.txt', '0-9.txt', '1-0.txt', '1-1.txt', '1-10.txt', '1-11.txt', '1-12.txt', '1-13.txt', '1-14.txt', '1-15.txt', '1-16.txt', '1-17.txt', '1-18.txt', '1-19.txt', '1-2.txt', '1-3.txt', '1-4.txt', '1-5.txt', '1-6.txt', '1-7.txt', '1-8.txt', '1-9.txt', '10-0.txt', '10-1.txt', '10-10.txt', '10-11.txt', '10-13.txt', '10-14.txt', '10-15.txt', '10-16.txt', '10-17.txt', '10-18.txt', '10-19.txt', '10-2.txt', '10-3.txt', '10-4.txt', '10-5.txt', '10-6.txt', '10-7.txt', '10-8.txt', '10-9.txt', '11-1.txt', '11-10.txt', '11-11.txt', '11-12.txt', '11-13.txt', '11-14.txt', '11-15.txt', '11-16.txt', '11-17.txt', '11-18.txt', '11-19.txt', '11-2.txt', '11-3.txt', '11-4.txt', '11-5.txt', '11-6.txt', '11-7.txt', '11-8.txt', '11-9.txt', '12-0.txt', '12-1.txt', '12-11.txt', '12-14.txt', '12-15.txt', '12-16.txt', '1